In [2]:
import netwolf_lib as nfb
import os
import socket
from math import ceil, log2
import threading
from time import sleep
from concurrent.futures import ThreadPoolExecutor
print('import library')

import library


In [7]:
test = True
def timer_out(*args):
    global test
    print('timer out:')
    for i in args:
        print('\t'+ str(i), end='')
    test = False
publish_time_of_directory = .05
t = threading.Timer(publish_time_of_directory, 
                    timer_out, [2, 4, 'time out'])
t.start()
t.isAlive()

while test:
    continue
    

timer out:
	2	4	time out

In [16]:
a =[(2,43,4), (3,4,5)]
for i, j, k, f in a:
    print(i, j, k, f)


ValueError: not enough values to unpack (expected 4, got 3)